In [19]:
import math
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from time import time
import csv
import os

In [20]:
folder_path = "../../data/preprocess_data/"
path = os.path.normpath(os.path.join(folder_path,'tap_squat_sai.mp4'))
path

'..\\..\\data\\preprocess_data\\tap_squat_sai.mp4'

In [21]:
# Khởi tạo Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Mở camera
cap = cv2.VideoCapture(path)  # Sử dụng camera mặc định (0) nếu có nhiều camera, thay bằng số khác

# Danh sách lưu toàn bộ landmarks
landmarks_list = []

# Khởi tạo đối tượng Pose từ Mediapipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    frame_count = 0  # Đếm số frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Tăng chỉ số frame
        frame_count += 1
        
        # Chuyển đổi ảnh sang RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Xử lý ảnh với Mediapipe Pose
        results = pose.process(image)
        
        # Chuyển lại ảnh sang BGR để hiển thị
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Lưu landmarks trong frame hiện tại
        if results.pose_landmarks:
            frame_landmarks = [frame_count]  # Lưu chỉ số frame đầu tiên
            for landmark in results.pose_landmarks.landmark:
                frame_landmarks.extend([landmark.x, landmark.y, landmark.z])  # Lưu x, y, z của mỗi landmark
            
            landmarks_list.append(frame_landmarks)  # Thêm landmarks vào danh sách

        # Hiển thị ảnh camera
        cv2.imshow('tap_squat_sai', image)
        
        # Nhấn 'q' để thoát
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Lưu dữ liệu landmarks vào file CSV khi thoát camera
with open('tap_squat_sai.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Ghi tiêu đề: Frame, Joint 1, Joint 2, ..., Joint 33
    header = ['Frame']
    for i in range(33):  # 33 landmarks
        header.extend([f'x_{i}', f'y_{i}', f'z_{i}'])
    writer.writerow(header)
    
    # Ghi dữ liệu từng frame
    writer.writerows(landmarks_list)

# Giải phóng camera và đóng cửa sổ
cap.release()
cv2.destroyAllWindows()

print("Landmarks và frame đã được lưu vào file tap_squat_dung.csv")


Landmarks và frame đã được lưu vào file tap_squat_dung.csv


In [22]:
import pandas as pd

In [23]:
df = pd.read_csv("tap_squat_sai.csv")
df

,Frame,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,...,z_29,x_30,y_30,z_30,x_31,y_31,z_31,x_32,y_32,z_32
0,1,0.498712,0.093197,-0.102076,0.503583,0.077471,-0.082601,0.506569,0.077926,-0.082601,...,0.162126,0.463557,0.724835,0.164921,0.538764,0.753499,0.046796,0.455329,0.753271,0.051181
1,2,0.498345,0.093197,-0.101903,0.503521,0.077496,-0.082890,0.506392,0.077939,-0.082891,...,0.148757,0.464678,0.724754,0.145457,0.542421,0.749586,0.040970,0.454961,0.753266,0.037246
2,3,0.498145,0.093191,-0.102375,0.503497,0.077502,-0.084418,0.506297,0.077940,-0.084420,...,0.146322,0.465399,0.724884,0.144085,0.543375,0.747647,0.039909,0.451912,0.752805,0.036623
3,4,0.498027,0.093190,-0.103417,0.503484,0.077514,-0.086188,0.506240,0.077947,-0.086190,...,0.146937,0.465858,0.724891,0.146811,0.543618,0.746600,0.040298,0.450082,0.752601,0.038990
4,5,0.497919,0.093171,-0.103392,0.503448,0.077534,-0.086316,0.506180,0.077953,-0.086318,...,0.142478,0.466566,0.725204,0.144277,0.543728,0.746300,0.035820,0.449925,0.752494,0.037225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276,2277,0.498371,0.090280,-0.250072,0.504402,0.077152,-0.235601,0.507633,0.077331,-0.235617,...,0.202166,0.467593,0.722890,0.216419,0.535522,0.748208,0.083020,0.451879,0.753284,0.100014
2277,2278,0.499068,0.081435,-0.222195,0.504922,0.068455,-0.206991,0.507828,0.068701,-0.207002,...,0.186088,0.467849,0.722737,0.196667,0.535068,0.747307,0.068636,0.451878,0.752077,0.081350
2278,2279,0.499286,0.081237,-0.225407,0.505087,0.067032,-0.210528,0.507910,0.067314,-0.210539,...,0.191837,0.467889,0.722739,0.199509,0.534475,0.746818,0.073929,0.451877,0.751543,0.083898
2279,2280,0.499480,0.080722,-0.213619,0.505305,0.066132,-0.197949,0.508019,0.066401,-0.197960,...,0.198549,0.467910,0.722596,0.200746,0.533825,0.746621,0.078772,0.452561,0.751467,0.082332
